# Imports

In [ ]:
import os
import math
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.transform import rowcol
from rasterio.plot import show
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import box, mapping
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from bokeh.plotting import figure, save
from bokeh.io import output_file as bokeh_output_file
from bokeh.models import FixedTicker, CustomJSTickFormatter
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool


# Inputs



DEMs downloaded from (not included in timberline_trail repo due to large size)

- https://pubs.oregon.gov/dogami/ldq/p-LDQ.htm

Timberline Trail SHP created by me from GPS data a long time ago. I no longer have the original GPX file, but they are widely available online and mine is in the timberline_trail repo.

- tlt.shp


# Constants

In [ ]:
# Adjust as needed after downloading DEMs
WORKING_DIR = r'C:\timberline_trail'

# DEM Processing

In [ ]:
def merge_raster_files(working_dir, file_paths, output_filename):
    """
    Merge multiple raster files into a single output file.

    Args:
    working_dir (str): The base directory for the raster files.
    file_paths (list): List of relative paths to the raster files.
    output_filename (str): Name of the output merged raster file.

    Returns:
    str: Path to the output merged raster file.
    """
    # Construct full file paths
    src_be_dem_files = [os.path.join(working_dir, path) for path in file_paths]

    # Check if all files exist
    for file_path in src_be_dem_files:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

    # Open all raster files and merge them
    with rasterio.open(src_be_dem_files[0]) as src_0:
        meta = src_0.meta.copy()
        with rasterio.Env():
            src_files = [src_0] + [rasterio.open(f) for f in src_be_dem_files[1:]]
            mosaic, out_trans = merge(src_files)

    # Update the metadata
    meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "compress": 'lzw',  
        "tiled": True,  
        "blockxsize": 256,  
        "blockysize": 256,  
    })

    # Write the merged raster
    output_path = os.path.join(working_dir, output_filename)
    with rasterio.open(output_path, "w", **meta) as dest:
        dest.write(mosaic)

    return output_path


# Make sure DEM file paths are configured correctly after downloading
file_paths = [
    r"LDQ-45121C6\LDQ-45121C6\2009_OLC_Hood to Coast\Bare_Earth\be45121c6\w001001.adf",
    r"LDQ-45121C7\LDQ-45121C7\2009_OLC_Hood to Coast\Bare_Earth\be45121c7a\w001001.adf",
    r"LDQ-45121D6\LDQ-45121D6\2009_OLC_Hood to Coast\Bare_Earth\be45121d6\w001001.adf",
    r"LDQ-45121D7\LDQ-45121D7\2009_OLC_Hood to Coast\Bare_Earth\be45121d7\w001001.adf",
]

try:
    output_file = merge_raster_files(WORKING_DIR, file_paths, "merged_dems.tif")
    print(f"Merged raster saved to: {output_file}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
def clip_raster_to_buffered_shapefile(
    working_dir,
    raster_filename,
    shapefile_filename,
    buffer_distance,
    output_filename,
):
    """
    Clip a raster to a buffered shapefile.

    Args:
    working_dir (str): The base directory for the input and output files.
    raster_filename (str): Name of the input raster file.
    shapefile_filename (str): Name of the input shapefile.
    buffer_distance (float): Distance to buffer the shapefile geometry
    (in units of the shapefile's CRS).
    output_filename (str): Name of the output clipped raster file.

    Returns:
    str: Path to the output clipped raster file.
    """
    raster_path = os.path.join(working_dir, raster_filename)
    shapefile_path = os.path.join(working_dir, shapefile_filename)
    output_path = os.path.join(working_dir, output_filename)

    # Open raster and assign its CRS
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs

        # Load the GeoDataFrame
        gdf = gpd.read_file(shapefile_path)

        # Make copy before buffering for later use
        tlt_gdf = gdf.copy(deep=True)

        # Ensure the GeoDataFrame is in the same CRS as the raster
        if gdf.crs != raster_crs:
            gdf = gdf.to_crs(raster_crs)

        # Buffer the geometry
        gdf['geometry'] = gdf['geometry'].buffer(buffer_distance)

        # Get the bounding box of the GeoDataFrame
        bbox = gdf.total_bounds

        # Create a rectangle from the bounding box
        bbox_polygon = box(*bbox)

        # Perform the clipping
        out_image, out_transform = mask(src, [mapping(bbox_polygon)], crop=True)

        # Update the metadata
        out_meta = src.meta.copy()
        out_meta.update(
            {
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "compress": 'lzw',  
                "tiled": True,  
                "blockxsize": 256,  
                "blockysize": 256,
            }
        )

    # Save the clipped raster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)

    return output_path, tlt_gdf


raster_file = os.path.join(WORKING_DIR, "merged_dems.tif")
tlt_shapefile = os.path.join(WORKING_DIR, "tlt.shp")
buffer_distance = 2000 
output_file = os.path.join(WORKING_DIR, "merged_dems_clipped.tif")

try:
    clipped_raster, tlt_gdf = clip_raster_to_buffered_shapefile(WORKING_DIR, raster_file, tlt_shapefile, buffer_distance, output_file)
    print(f"Clipped raster saved to: {clipped_raster}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Let's peek at our Timberline Trail gdf
tlt_gdf.head()

In [ ]:
def hillshade(array, azimuth, angle_altitude):
    """
    Calculate hillshade from a DEM (Digital Elevation Model) array.

    Args:
    array (numpy.ndarray): Input DEM array.
    azimuth (float): Azimuth angle in degrees (direction of light source).
    angle_altitude (float): Altitude angle in degrees (height of light source).

    Returns:
    numpy.ndarray: Hillshade array.
    """
    # Adjust azimuth for internal calculations
    azimuth = 360.0 - azimuth

    # Calculate gradient of the DEM in x and y directions
    # This gives us the rate of change of elevation in both directions
    x, y = np.gradient(array.astype(np.float64))

    # Calculate slope
    # np.clip is used to avoid taking square root of negative numbers
    # slope is the angle of inclination to the horizontal plane
    slope = np.pi / 2.0 - np.arctan(np.sqrt(np.clip(x * x + y * y, 0, None)))

    # Calculate aspect (direction of slope)
    # Aspect is the direction of the steepest descent
    aspect = np.arctan2(-x, y)

    # Convert azimuth and altitude angles to radians
    azimuthrad = azimuth * np.pi / 180.0
    altituderad = angle_altitude * np.pi / 180.0

    # Calculate hillshade
    # This formula simulates the illumination conditions of the surface
    shaded = np.sin(altituderad) * np.sin(slope) + np.cos(altituderad) * np.cos(
        slope
    ) * np.cos((azimuthrad - np.pi / 2.0) - aspect)

    # Ensure shaded values are within the range [-1, 1]
    shaded = np.clip(shaded, -1, 1)

    # Scale and convert the shaded relief to 0-255 (8-bit) range
    return (255 * (shaded + 1) / 2).astype(np.uint8)


def create_hillshade(
    input_raster_path, output_hillshade_path, azimuth=315, angle_altitude=45
):
    """
    Create a hillshade raster from an input DEM raster.

    Args:
    input_raster_path (str): Path to the input DEM raster file.
    output_hillshade_path (str): Path to save the output hillshade raster file.
    azimuth (float): Azimuth angle for hillshade calculation (default: 315).
    angle_altitude (float): Altitude angle for hillshade calculation
    (default: 45).

    Returns:
    str: Path to the output hillshade raster file.
    """
    # Open the input raster
    with rasterio.open(input_raster_path) as src:
        elevation = src.read(1).astype(
            np.float64
        )  # Convert to float64 for higher precision
        profile = src.profile

    # Create hillshade
    hs_array = hillshade(elevation, azimuth, angle_altitude)

    # Update the profile for the new hillshade raster
    # Remove nodata value for uint8 output
    profile.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw',
        nodata=None,  
    )

    # Save the hillshade as a new TIFF
    with rasterio.open(output_hillshade_path, 'w', **profile) as dst:
        dst.write(hs_array, 1)

    return output_hillshade_path


input_raster = os.path.join(WORKING_DIR, 'merged_dems_clipped.tif')
output_hillshade = os.path.join(WORKING_DIR, 'merged_dems_clipped_hs.tif')

try:
    hillshade_path = create_hillshade(input_raster, output_hillshade)
    print(f"Hillshade raster saved to: {hillshade_path}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Timberline Trail shapefile and gdf processing

In [ ]:
def get_polyline_end_coords(geom):
    """
    Extract the start and end coordinates of a polyline geometry.

    This function takes a shapely LineString or MultiLineString geometry and
    returns a list containing the coordinates of its start and end points.

    Args:
        geom (shapely.geometry.LineString or shapely.geometry.MultiLineString): 
            The input polyline geometry.

    Returns:
        list: A list containing two tuples. The first tuple represents the 
              start coordinates (x, y), and the second tuple represents the 
              end coordinates (x, y) of the polyline.

    Note:
        For MultiLineString geometries, it returns the start of the first 
        component and the end of the last component.
    """
    return list(geom.coords)

def add_lat_lon_cols(gdf):
    """
    Add start and end latitude and longitude columns to a GeoDataFrame.

    This function takes a GeoDataFrame containing polyline geometries and adds
    four new columns: 'start_lat.', 'start_lon.', 'end_lat.', and 'end_lon.'.
    These columns contain the latitude and longitude of the start and end 
    points of each polyline.

    Args:
        gdf (geopandas.GeoDataFrame): The input GeoDataFrame containing polyline
                                      geometries in its 'geometry' column.

    Returns:
        geopandas.GeoDataFrame: The input GeoDataFrame with four new columns added:
                                'start_lat.', 'start_lon.', 'end_lat.', 'end_lon.'

    Note:
        This function assumes that the geometries in the GeoDataFrame are 
        LineString or MultiLineString objects. It will use the first coordinate
        as the start point and the last coordinate as the end point for each geometry.
    """
    coords = gdf.apply(lambda row: get_polyline_end_coords(row.geometry), axis=1)
    gdf['start_lat.'] = [x[0][1] for x in coords]
    gdf['start_lon.'] = [x[0][0] for x in coords]
    gdf['end_lat.'] = [x[-1][1] for x in coords] 
    gdf['end_lon.'] = [x[-1][0] for x in coords]  
    
    return gdf

try:
    tlt_gdf = add_lat_lon_cols(tlt_gdf)
    print(f"Latitude and Longitude data added to tlt_gdf")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
tlt_gdf.head()

In [ ]:
def get_elevation(x, y):
    """
    Get the elevation value for a given coordinate.

    This function takes an x and y coordinate, converts it to a row and column
    in the raster, and returns the corresponding elevation value.

    Args:
        x (float): The x-coordinate (typically longitude).
        y (float): The y-coordinate (typically latitude).

    Returns:
        float: The elevation value at the given coordinate. Returns np.nan if
               the coordinate is outside the raster bounds or if the value
               is equal to the nodata value.

    Note:
        This function assumes that 'raster_transform', 'elevation_data', and
        'nodata_value' are defined in the global scope or passed as parameters.
    """
    # Convert geographic coordinates to raster row and column
    row, col = rowcol(raster_transform, x, y)

    # Check if the resulting row and column are within the raster bounds
    if (
        0 <= row < elevation_data.shape[0]
        and 0 <= col < elevation_data.shape[1]
    ):
        elev = elevation_data[row, col]
        # Return the elevation if not the nodata value, otherwise return np.nan
        return elev if elev != nodata_value else np.nan
    else:
        # Return np.nan if the coordinate is outside the raster bounds
        return np.nan


def get_start_end_elevations(geometry):
    """
    Get the start and end elevations for a LineString or MultiLineString
    geometry.

    This function extracts the start and end points of the given geometry and
    retrieves their respective elevation values.

    Args:
        geometry (shapely.geometry.LineString or
        shapely.geometry.MultiLineString):
            The input line geometry.

    Returns:
        tuple: A tuple containing two float values:
               - The elevation at the start point of the geometry.
               - The elevation at the end point of the geometry.
               Returns (np.nan, np.nan) if the geometry is neither a LineString
               nor a MultiLineString.

    Note:
        This function relies on the get_elevation function to retrieve
        elevation values.
    """
    if geometry.geom_type == 'LineString':
        # For LineString, use the first and last coordinates
        start_point = geometry.coords[0]
        end_point = geometry.coords[-1]
    elif geometry.geom_type == 'MultiLineString':
        # For MultiLineString, use the start of the first line and end of the
        # last line
        start_point = geometry[0].coords[0]
        end_point = geometry[-1].coords[-1]
    else:
        # Return np.nan for both start and end if geometry is neither
        # LineString nor MultiLineString
        return np.nan, np.nan

    # Get elevation for start and end points
    start_elev = get_elevation(start_point[0], start_point[1])
    end_elev = get_elevation(end_point[0], end_point[1])

    return start_elev, end_elev


try:
    with rasterio.open(
        os.path.join(WORKING_DIR, 'merged_dems_clipped.tif')
    ) as src:
        elevation_data = src.read(1)
        raster_transform = src.transform
        raster_crs = src.crs
        nodata_value = src.nodata

    if tlt_gdf.crs != raster_crs:
        tlt_gdf = tlt_gdf.to_crs(raster_crs)

    tlt_gdf['str_pt_z'], tlt_gdf['end_pt_z'] = zip(
        *tlt_gdf.geometry.apply(get_start_end_elevations)
    )
    print(f"Elevation data added to tlt_gdf")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
tlt_gdf.head()

In [ ]:
def apply_formatting(tlt_gdf):
    """
    Apply formatting to a GeoDataFrame containing trail segments.
    
    This function adds and modifies several columns in the input GeoDataFrame:
    - Formats 'seg_no' as a zero-padded 3-digit string
    - Sorts the DataFrame by 'seg_no'
    - Calculates segment length and elevation change
    - Rounds numerical values
    
    Args:
    tlt_gdf (gpd.GeoDataFrame): Input GeoDataFrame containing trail segments
    
    Returns:
    gpd.GeoDataFrame: Formatted GeoDataFrame with additional columns and modifications
    """
    # Create a copy of the input GeoDataFrame to avoid modifying the original
    tlt_gdf = tlt_gdf.copy()
    
    # Format seg_no as zero-padded 3-digit string
    tlt_gdf['seg_no'] = tlt_gdf['seg_no'].map(lambda x: f'{x:0>3}')
    
    # Sort the DataFrame by seg_no
    tlt_gdf.sort_values('seg_no', inplace=True)
    
    # Calculate length and elevation change
    tlt_gdf['length'] = tlt_gdf.geometry.length
    tlt_gdf['elev_ch'] = tlt_gdf['end_pt_z'] - tlt_gdf['str_pt_z']
    
    # Add new columns
    tlt_gdf['slope_per'] =  (tlt_gdf['elev_ch'] / tlt_gdf['length'] *100).round(1)
    
    # Round numerical values
    columns_to_round = ['length', 'str_pt_z', 'end_pt_z', 'elev_ch']
    tlt_gdf[columns_to_round] = tlt_gdf[columns_to_round].round(1)
    
    return tlt_gdf

try:
    tlt_gdf = apply_formatting(tlt_gdf)
    print(f"Formatting applied to tlt_gdf")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
tlt_gdf.head()

# Bokeh elevation profile processing

In [ ]:
def prepare_elevation_data(tlt_gdf):
    """
    Prepare elevation data for plotting.

    Args:
        tlt_gdf (geopandas.GeoDataFrame): GeoDataFrame containing trail
        segment data.

    Returns:
        tuple:
            - x_cumsum (list): Cumulative distances along the trail.
            - z_list (list): Elevations at each point.
            - bokeh_coord_list (list): Coordinates formatted for plotting.
    """
    # Calculate cumulative distances, adding 0 at the start
    x_cumsum = [0] + tlt_gdf['length'].cumsum().tolist()

    # Create elevation list, adding the first elevation at the end
    z_list = tlt_gdf['str_pt_z'].tolist() + [tlt_gdf['str_pt_z'].iloc[0]]

    # Create coordinate list for plotting
    bokeh_coord_list = []
    for i in range(len(x_cumsum) - 1):
        bokeh_coord_list.append(
            ((x_cumsum[i], x_cumsum[i + 1]), (z_list[i], z_list[i + 1]))
        )

    return x_cumsum, z_list, bokeh_coord_list


def create_color_palette(tlt_gdf, num_colors):
    """
    Create a color palette based on slope percentages.

    Args:
        tlt_gdf (geopandas.GeoDataFrame): GeoDataFrame containing trail
        segment data.
        num_colors (int): Number of colors in the palette.

    Returns:
        list: List of hex color codes.
    """
    # Create a color palette
    palette = sns.color_palette("RdYlGn_r", num_colors)
    hex_colors = [mcolors.rgb2hex(rgb) for rgb in palette]

    # Sort the GeoDataFrame by slope percentage and assign colors
    tlt_gdf_sort_slope_per = tlt_gdf.sort_values('slope_per')
    tlt_gdf_sort_slope_per['sns_color'] = hex_colors

    # Sort back to original order
    color_sort_gdf = tlt_gdf_sort_slope_per.sort_index()

    return color_sort_gdf['sns_color'].tolist()


def create_elevation_profile(
    x_cumsum, z_list, bokeh_coord_list, colors, tlt_gdf
):
    """
    Create and save an elevation profile plot using Bokeh.

    Args:
        x_cumsum (list): Cumulative distances along the trail.
        z_list (list): Elevations at each point.
        bokeh_coord_list (list): Coordinates formatted for Bokeh plotting.
        colors (list): List of colors for each trail segment.
        tlt_gdf (geopandas.GeoDataFrame): GeoDataFrame containing trail
        segment data.
    """

    # Round x-axis labels
    x_cumsum_labs = [round(x, 1) for x in x_cumsum]

    # Create Bokeh figure
    p = figure(
        width=1800,
        height=300,
        x_range=(x_cumsum_labs[0], x_cumsum_labs[-1]),
        y_range=(min(z_list) - 100, max(z_list) + 100),
        tools="hover,pan,wheel_zoom,box_zoom,reset,save",
        title="Timberline Trail Elevation Profile"
    )

    # Add hover tool
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [
        ("Distance", "$x{0,0.0} ft"),
        ("Elevation", "$y{0,0.0} ft"),
        ("Segment Length", "@length{0,0.0} ft"),
        ("Elevation Change", "@elev_ch{0,0.0} ft"),
        ("Slope", "@slope_per{0.0}%"),
    ]
    hover.mode = 'vline'  # Use vertical line mode for hover tool

    # Plot each segment with its corresponding color and data for tooltips
    for i, (coords, color) in enumerate(zip(bokeh_coord_list, colors)):
        source = ColumnDataSource(
            data=dict(
                x=coords[0],
                y=coords[1],
                length=[tlt_gdf.iloc[i]['length']] * 2,
                elev_ch=[tlt_gdf.iloc[i]['elev_ch']] * 2,
                slope_per=[tlt_gdf.iloc[i]['slope_per']] * 2,
            )
        )
        p.line('x', 'y', line_width=2, line_color=color, source=source)

    # Configure x-axis
    p.xaxis.ticker = FixedTicker(ticks=x_cumsum_labs)
    p.xaxis.formatter = CustomJSTickFormatter(code="return tick.toFixed(1)")
    p.xaxis.major_label_orientation = math.pi / 2
    p.xaxis.axis_label_text_font_size = "8pt"
    p.xaxis.major_label_text_font_size = "8pt"
    p.xaxis.axis_label = 'Distance (ft)'

    # Configure y-axis
    p.yaxis.axis_label_text_font_size = "8pt"
    p.yaxis.major_label_text_font_size = "8pt"
    p.yaxis.axis_label = 'Elevation (ft)'

    bokeh_output_file(os.path.join(WORKING_DIR, "elevation_profile.html"))
    save(p)


def generate_elevation_profile(tlt_gdf):
    """
    Generate an elevation profile from trail data.

    Args:
        tlt_gdf (geopandas.GeoDataFrame):  trail segment GeoDataFrame.

    Returns:
        list: List of colors used in the elevation profile.
    """
    # Prepare data for plotting
    x_cumsum, z_list, bokeh_coord_list = prepare_elevation_data(tlt_gdf)

    # Create color palette
    colors = create_color_palette(tlt_gdf, 36)

    # Create and save elevation profile
    create_elevation_profile(
        x_cumsum, z_list, bokeh_coord_list, colors, tlt_gdf
    )

    return colors




try:
    colors = generate_elevation_profile(tlt_gdf)
    print(f"Bokeh TLT elevation profle saved to: {WORKING_DIR}\elevation_profile.html")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# let's peek at our sorted hex colors used to convey segment difficulty

"""

colors = ['#fed481',
 '#b3df72',
 '#93d168',
 '#1e9a51',
 '#f16640',
 '#feea9b',
 '#f88950',
 '#bfe47a',
 '#06733d',
 '#0d8044',
 '#82c966',
 '#fdbb6c',
 '#cc2627',
 '#fff2aa',
 '#fafdb8',
 '#f57748',
 '#fa9b58',
 '#eff8aa',
 '#70c164',
 '#fdad60',
 '#e5f49b',
 '#a5d86a',
 '#e95538',
 '#d93429',
 '#148e4b',
 '#33a456',
 '#daf08d',
 '#fdc776',
 '#5db961',
 '#be1827',
 '#fee18d',
 '#fffbb8',
 '#cdea83',
 '#48ae5c',
 '#b10b26',
 '#e14430']
 
""";

In [ ]:
# Let's save a new SHP with the the colored segments
tlt_gdf_colors = tlt_gdf.copy(deep=True)
tlt_gdf_colors['colors'] = colors
tlt_gdf_colors.to_file(os.path.join(WORKING_DIR, 'tlt_gdf_colors.shp'))

In [ ]:
tlt_gdf

# Segment difficulty styled gdf processing

In [ ]:
def rename_columns(df):
    """
    Rename columns of the DataFrame for better readability.

    This function maps original column names to more descriptive and readable names.

    Args:
        df (pandas.DataFrame): The input DataFrame with original column names.

    Returns:
        pandas.DataFrame: A DataFrame with renamed columns.
    """
    column_map = {
        'seg_no': 'segment',
        'length_m': 'length',
        'str_pt_lon': 'start lon.',
        'str_pt_lat': 'start lat.',
        'str_pt_z': 'start elev.',
        'end_pt_lon': 'end lon.',
        'end_pt_lat': 'end lat.',
        'end_pt_z': 'end elev.',
        'elev_ch': 'elev. change',
        'slope_per': 'slope per.'
    }
    return df.rename(columns=column_map)

def style_trail_data(trail_df, colors):
    """
    Apply styling to the trail data DataFrame with correct rounding using Pandas styling.

    This function applies color highlighting to specific columns and formats numeric columns
    with appropriate decimal places.

    Args:
        trail_df (pandas.DataFrame): The input DataFrame containing trail data.
        colors (list): A list of colors to be used for highlighting.

    Returns:
        pandas.io.formats.style.Styler: A styled DataFrame.
    """
    def highlight_cols(s):
        """
        Helper function to apply background colors to specific columns.
        """
        if s.name in ['elev. change', 'slope per.']:
            return [f'background-color: {color}' for color in colors]
        return [''] * len(s)

    # Apply highlighting
    styled = trail_df.style.apply(highlight_cols, axis=0)
    
    # Format latitude and longitude columns to 6 decimal places
    lat_lon_cols = ['start lat.', 'start lon.', 'end lat.', 'end lon.']
    styled = styled.format({col: '{:.6f}' for col in lat_lon_cols})
    
    # Format other numeric columns to 1 decimal place
    numeric_cols = ['start elev.', 'end elev.', 'length', 'distance', 'elev. change', 'slope per.']
    styled = styled.format({col: '{:.1f}' for col in numeric_cols})
    
    return styled

def prepare_trail_data(trail_df, colors):
    """Prepare and format the trail data."""
    formatted_df = trail_df.iloc[:, 2:].copy()
    formatted_df['colors'] = colors
    formatted_df = rename_columns(formatted_df)
    formatted_df = formatted_df.iloc[:, :-1]
    formatted_df.insert(0, 'segment', trail_df['segment'])
    formatted_df.insert(7, 'distance', formatted_df['length'].cumsum())
    return formatted_df


def process_trail_data(trail_df, colors):
    """
    Main function to process and style trail data.

    This function orchestrates the entire process of preparing and styling the trail data.

    Args:
        trail_df (pandas.DataFrame): The input DataFrame containing raw trail data.
        colors (list): A list of colors to be used for highlighting in the styled output.

    Returns:
        tuple: A tuple containing two elements:
               - formatted_df (pandas.DataFrame): The processed DataFrame with renamed columns.
               - styled_df (pandas.io.formats.style.Styler): The styled version of the formatted DataFrame.

    """
    # Prepare the trail data (rename columns, add calculated fields, etc.)
    formatted_df = prepare_trail_data(trail_df, colors)
    
    # Apply styling to the formatted DataFrame
    styled_df = style_trail_data(formatted_df, colors)
    
    return formatted_df, styled_df

try:
    tlt_gdf = rename_columns(tlt_gdf)
    formatted_df, styled_df = process_trail_data(tlt_gdf, colors)
    print(f"Styling applied to tlt_gdf to create styled_df")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Here's our table to accompany the hillshade, difficulty colored Timberline Trail shapefile, and Bokeh elevation profile
styled_df